In [3]:
import pybithumb as pb

In [4]:
tickers=pb.get_tickers()
pb.get_current_price('BTC')

62177000.0

In [7]:
len(tickers)-40

142

In [8]:
import time
for i in range(len(tickers)-176):
    price=pb.get_current_price(tickers[i])
    print(tickers[i],price)
    time.sleep(0.1)

BTC 62130000.0
ETH 4946000.0
LTC 196900.0
ETC 45640.0
XRP 1110.0
BCH 552000.0


In [12]:
detail=pb.get_market_detail('BTC')
print(detail)

(61858000.0, 62444000.0, 61564000.0, 62252000.0, 100.55893414)


In [17]:
order=pb.get_orderbook('btc')
print(order.keys())

dict_keys(['timestamp', 'payment_currency', 'order_currency', 'bids', 'asks'])


In [23]:
import datetime
ms=int(order['timestamp'])
print(datetime.datetime.fromtimestamp(ms/1000))

2021-12-24 16:54:53.167000


In [26]:
bids=order['bids']
for bid in bids:
    print('매수호가:',bid['price'])
    print('수량:',bid['quantity'])

매수호가: 62150000.0
수량: 0.0105
매수호가: 62148000.0
수량: 0.2721
매수호가: 62147000.0
수량: 0.0408
매수호가: 62129000.0
수량: 0.4573
매수호가: 62128000.0
수량: 0.0189


In [3]:
import pybithumb as pb
all=pb.get_current_price('ALL')
for ticker,data in all.items():
    print(ticker,data['closing_price'])

BTC 62599000
ETH 4982000
LTC 198300
ETC 46030
XRP 1135
BCH 555000
QTUM 12140
BTG 54050
EOS 4195
ICX 1760
TRX 99.38
ELF 487.9
OMG 8320
KNC 1711
GLM 587.7
ZIL 88.9
WAXP 605.5
POWR 581.5
LRC 2757
STEEM 558
STRAX 1875
ZRX 1057
REP 22080
XEM 168.2
SNT 90.9
ADA 1730
CTXC 572.8
BAT 1599
WTC 1222
THETA 5755
LOOM 113.9
WAVES 19860
TRUE 263
LINK 27160
ENJ 3454
VET 115.1
MTL 2820
IOST 42.65
TMTG 4.55
QKC 28.84
ATOLO 76.17
AMO 3.234
BSV 159700
ORBS 109.8
TFUEL 267.8
VALOR 1650
CON 12.01
ANKR 138.7
MIX 25.36
CRO 817
FX 1134
CHR 953.9
MBL 11
MXC 64.07
FCT 178.2
TRV 30
DAD 230.2
WOM 227.7
SOC 19.1
EM 14.5
BOA 269.9
FLETA 38.6
SXP 1998
COS 34.31
APIX 112.4
EL 13.63
BASIC 9.566
HIVE 1991
XPR 23.65
VRA 52.87
FIT 3.407
EGG 14.15
BORA 1337
ARPA 133.4
CTC 3165
APM 41.77
CKB 26.4
AERGO 341.3
ANW 50.85
CENNZ 214.6
EVZ 105.7
CYCLUB 157.9
SRM 4995
QTCON 15.81
UNI 21930
YFI 37577000
UMA 11980
AAVE 336600
COMP 277400
REN 800
BAL 22810
RSR 41.19
NMR 41430
RLC 3842
UOS 2191
SAND 7390
GOM2 60
RINGX 73.64
BEL 1981
O

In [34]:
btc=pb.get_ohlcv('BTC')

In [47]:
def bullbear(ticker):
    df=pb.get_ohlcv(ticker)
    close=df['close']
    ma10=close.rolling(10).mean()
    last_ma10=ma10[-2]
    cur=pb.get_current_price(ticker)
    if last_ma10<cur:
        print(ticker,': Bull')
    else:
        print(ticker,': Bear')

In [42]:
bullbear('BTC')

Bull


In [48]:
tickers=pb.get_tickers()
tickers[:10]

['BTC', 'ETH', 'LTC', 'ETC', 'XRP', 'BCH', 'QTUM', 'BTG', 'EOS', 'ICX']

In [49]:
for ticker in tickers[:10]:
    bullbear(ticker)

BTC : Bull
ETH : Bull
LTC : Bull
ETC : Bull
XRP : Bull
BCH : Bull
QTUM : Bull
BTG : Bull
EOS : Bull
ICX : Bull


In [1]:
import sys
import pybithumb as pb
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtCore import *
form_class = uic.loadUiType("bull.ui")[0]
tickers = ["BTC", "ETH", "BCH", "ETC"]
class Worker(QThread):
    finished=pyqtSignal(dict)
    def run(self):
        while True:
            data={}
            for ticker in tickers:
                data[ticker]=self.bullbear(ticker)
            self.finished.emit(data)
            self.msleep(500)
    def bullbear(self,ticker):
        df=pb.get_ohlcv(ticker)
        close=df['close']
        ma10=close.rolling(10).mean()
        last_ma10=ma10[-2]
        cur=pb.get_current_price(ticker)
        if last_ma10<cur:
            state='Bull'
        else:
            state='Bear'
        return cur,last_ma10,state
class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.worker=Worker()
        self.worker.finished.connect(self.update_table_widget)
        self.worker.start()
    
    
    def update_table_widget(self,data):
        for ticker,info in data.items():
            i=tickers.index(ticker)
            self.tableWidget.setItem(i,0,QTableWidgetItem(ticker))
            self.tableWidget.setItem(i,1,QTableWidgetItem(str(info[0])))
            self.tableWidget.setItem(i,2,QTableWidgetItem(str(info[1])))
            self.tableWidget.setItem(i,3,QTableWidgetItem(str(info[2])))
app = QApplication(sys.argv)
win = MyWindow()
win.show()
app.exec_()

0

In [2]:
QTableWidgetItem('BTC')